<a href="https://colab.research.google.com/github/AseelAbuAl-Rub/NLP-Project/blob/main/Task_1%2C2_and_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this code, we load the data from google drive and stored it in data frames for each file then concat train negative and positive in one data frame name: train data, also concat test in one data frame name: test data frame; convert the data type of labelling to boolean. We apply a cleaning process: remove numbers, special characters, and punctuation marks then tokenized text, removed stop words and apply Lemmatization.
As a embedding representation text, we used CBOW Word2vec to build model by using training data, then tested the test data on the trained model. Logistic regression machine learning used to predict the label.

In [ ]:
!pip install pydrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
import nltk
import re
import numpy as np
from nltk.tokenize import word_tokenize,sent_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models import Word2Vec
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#@title Authenticate & authorize with google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#@title Sets up a local directory 

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

#    https://developers.google.com/drive/v2/web/search-parameters
train_neg_id = '1PXqBtQRUb4gdoSbzz-kUEXq7F9O6wTPz'
train_pos_id = '1i_m3qQC3OPAUckbljbiZ-6HfTQHPrNyh'

test_neg_id = '1qQD54k2x42mbzAKRrQkL4GhdtRgGVRAK'
test_pos_id = '1yTaxzpANeSaSk7vwCIEByRJmzEqhoIcz'

local_download_path = os.path.expanduser('~/data')

In [ ]:
#@title Authenticate & authorize with google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#@title Read from drive and store in dataframe function
def read_from_drive(drive, parent_folder_id, local_download_path):
    df = pd.DataFrame()  # Instantiate an empty DataFrame
    file_list = drive.ListFile({'q': f"'{parent_folder_id}' in parents"}).GetList()

    for f in file_list:
        fname = os.path.join(local_download_path, f['title'])
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)
        
        with open(fname, 'r') as file:
            file_content = file.read()
            df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
    
    return df

In [ ]:
#@title Apply (read from drive and store in df)function
train_neg = read_from_drive(drive, train_neg_id, local_download_path)
train_neg['flag'] = 0
train_pos = read_from_drive(drive, train_pos_id, local_download_path)
train_pos['flag'] = 1

test_neg = read_from_drive(drive, test_neg_id, local_download_path)
test_neg['flag'] = 0
test_pos = read_from_drive(drive, test_pos_id, local_download_path)
test_pos['flag'] = 1

test_neg

<ipython-input-7-915ddb26fda4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-7-915ddb26fda4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-7-915ddb26fda4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-7-915ddb26fda4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-7

,Tweets,flag
0,Widow hires a psychopath as a handyman. Sloppy...,0
1,The only reason this movie is not given a 1 (a...,0
2,The most positive points in this film were the...,0
3,"Beware, My Lovely (1952) Dir: Harry Horner <br...",0
4,"I quite enjoyed The Wrecking Crew (1999), whic...",0
...,...,...
91,Bloody Birthday opens to a shot of Meadowvale ...,0
92,"A mean spirited, repulsive horror film about 3...",0
93,"Wow, another Kevin Costner hero movie. Postman...",0
94,Once again Mr. Costner has dragged out a movie...,0


In [ ]:
#@title Concat the data
train_data = pd.concat([train_neg, train_pos], ignore_index=True)
test_data = pd.concat([test_neg, test_pos], ignore_index=True)

In [ ]:
#@title Stop words
#Stop Words
stop_words=set(stopwords.words('english'),)

In [ ]:
#@title Remove Characters And Number Function 
def remove_num_special_char(text):
    cleaned_text = re.sub(r"[^a-zA-Z\s]", "", text)
    return cleaned_text



In [ ]:
#@title Remove Stop Words & Lemmatization Function
def remove_stopwords_lem(text):
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(token) for token in text if token not in stop_words]
    return filtered_tokens
    

In [ ]:
#@title Apply (remove character and num) function

train_data['cleaned_text'] = train_data['Tweets'].apply(remove_num_special_char)

test_data['cleaned_text'] = test_data['Tweets'].apply(remove_num_special_char)

In [ ]:
#@title Tokenize text
train_data['tokenized_text'] = train_data['cleaned_text'].apply(lambda x: word_tokenize(x))

test_data['tokenized_text'] = test_data['cleaned_text'].apply(lambda x: word_tokenize(x))

In [ ]:
#@title Convert to lower case 
train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda tokens: [token.lower() for token in tokens])

test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda tokens: [token.lower() for token in tokens])

In [ ]:
#@title Apply (remove stop words & Lem) function
train_data['cleaned_tokens'] = train_data['tokenized_text'].apply(remove_stopwords_lem)

test_data['cleaned_tokens'] = test_data['tokenized_text'].apply(remove_stopwords_lem)

In [ ]:
#@title Word2vec
sentences = train_data['cleaned_tokens'].values.tolist()
train_data_model = Word2Vec(sentences=sentences, vector_size=200, window=3, min_count=1, workers=4)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Vectorize the sentences using the Word2Vec model
vectorized_sentences = []
for sentence in train_data['cleaned_tokens']:
    sentence_vectors = [train_data_model.wv[word] for word in sentence if word in train_data_model.wv]
    if sentence_vectors:
        sentence_vector = np.mean(sentence_vectors, axis=0)
        vectorized_sentences.append(sentence_vector)
    else:
        vectorized_sentences.append(np.zeros(train_data_model.vector_size))

# Convert the list of vectors to a numpy array
X_train = np.array(vectorized_sentences)

# Assign labels to the sentences based on your classification task
y_train = train_data['flag']

# Vectorize the sentences in the test data
vectorized_test_sentences = []
for sentence in test_data['cleaned_tokens']:
    sentence_vectors = [train_data_model.wv[word] for word in sentence if word in train_data_model.wv]
    if sentence_vectors:
        sentence_vector = np.mean(sentence_vectors, axis=0)
        vectorized_test_sentences.append(sentence_vector)
    else:
        vectorized_test_sentences.append(np.zeros(train_data_model.vector_size))

# Convert the list of vectors to a numpy array
X_test = np.array(vectorized_test_sentences)

# Assign labels to the sentences in the test data
y_test = test_data['flag']

# Train a logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)

train_data_model.save("trained_word2vec.model")

accuracy


0.4824120603015075